In [1]:
import pandas as pd

from utils.tradingview import TradingView

In [2]:
df = TradingView.get_base_symbols()

In [3]:
from utils.industry import get_classification


def merge_nse_industry_classification():
    classification_df = get_classification()
    merged = df.copy().merge(classification_df, left_index=True, right_index=True, how='left', suffixes=('', '_new'))

    # Loop through columns in df2
    for col in classification_df.columns:
        new_col = f'{col}_new'
        if col in df.columns:
            # Overwrite only where df2 has non-null values
            merged[col] = merged[new_col].combine_first(merged[col])
            # Drop the extra merged column
            merged.drop(columns=[new_col], inplace=True)
        else:
            # New column, already added by merge, nothing to do
            pass

    return merged

OCI FS Configured


In [4]:
df = merge_nse_industry_classification()

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 2812 entries, NSE:RELIANCE to NSE:GSMFOILS
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype        
---  ------                                 --------------  -----        
 0   name                                   2812 non-null   object       
 1   type                                   2812 non-null   category     
 2   isin                                   2812 non-null   object       
 3   description                            2812 non-null   object       
 4   logo                                   2812 non-null   object       
 5   pricescale                             2812 non-null   int16        
 6   minmov                                 2812 non-null   int16        
 7   currency                               2812 non-null   category     
 8   fundamental_currency                   2812 non-null   category     
 9   market                                 2812 non-null   categ

In [6]:
from utils.fundamentals import get_fundamentals
funda_df = get_fundamentals()

In [7]:
funda_df

,quarterly,yearly,revenue_fy_1,revenue_fy_2,revenue_fy_3,revenue_fy_4,revenue_fy_5,revenue_growth_fy_1,revenue_growth_fy_2,revenue_growth_fy_3,...,eps_growth_yoy_fq_3,eps_growth_yoy_fq_4,eps_growth_yoy_fq_5,eps_growth_yoy_fq_6,eps_growth_qoq_fq_1,eps_growth_qoq_fq_2,eps_growth_qoq_fq_3,eps_growth_qoq_fq_4,eps_growth_qoq_fq_5,eps_growth_qoq_fq_6
ticker,,,,,,,,,,,,,,,,,,,,,
NSE:RELIANCE,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",964693.00,901064.00,877835.00,695963.00,466924.0,7.06,2.65,26.13,...,-4.82,4.19,8.19,19.70,4.67,11.93,9.38,-20.13,9.80,-0.78
NSE:HDFCBANK,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",336367.43,283649.02,170754.05,135936.41,128552.4,18.59,66.12,25.61,...,5.22,-2.17,2.75,-0.18,6.53,-1.24,7.98,-6.59,2.02,2.25
NSE:TCS,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",255324.00,240893.00,225458.00,191754.00,164177.0,5.99,6.85,17.58,...,6.19,9.98,10.37,2.19,-1.23,3.92,-1.08,-3.17,13.47,-2.29
NSE:AIRTELPP.E1,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NSE:BHARTIARTL,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",162708.10,149982.40,139144.80,116546.90,100615.8,8.48,7.79,19.39,...,153.87,-31.89,51.96,-37.89,311.27,-13.87,99.72,-15.46,80.93,-16.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NSE:GOLDCASE,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NSE:GSEC10IETF,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NSE:WINNY,"[[Date, Revenue, Revenue Growth YoY, Revenue G...","[[Date, Revenue, Revenue Growth, Expenses, Emp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from utils.fundamentals import get_fundamentals


def merge_fundamentals():
    funda_df = get_fundamentals()
    merged = df.copy().merge(funda_df, left_index=True, right_index=True, how='left', suffixes=('', '_new'))

    # Loop through columns in df2
    for col in funda_df.columns:
        new_col = f'{col}_new'
        if col in df.columns:
            # Overwrite only where df2 has non-null values
            merged[col] = merged[new_col].combine_first(merged[col])
            # Drop the extra merged column
            merged.drop(columns=[new_col], inplace=True)
        else:
            # New column, already added by merge, nothing to do
            pass

    return merged

In [9]:
df = merge_fundamentals()

In [10]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 2812 entries, NSE:RELIANCE to NSE:GSMFOILS
Columns: 158 entries, name to eps_growth_qoq_fq_6
dtypes: category(8), datetime64[s](2), float64(137), int16(2), object(9)
memory usage: 5.2 MB
